## Comparative Analysis of Option Pricing Methods

In this notebook, we benchmark and compare three key option pricing approaches for European vanilla options:

- **Black-Scholes-Merton (BSM)** closed-form analytical formula
- **Monte Carlo Simulation** with Euler discretization
- **PDE Solver** using Crank-Nicolson finite difference method

### Objectives:
- Compare prices across all methods for consistency
- Measure runtime and performance
- Analyze convergence trends with varying grid/path size

In [7]:
import numpy as np
import time
import matplotlib.pyplot as plt
import os, sys

sys.path.append(os.path.abspath("../../"))

from pricing.bsm import black_scholes_price
from pricing.monte_carlo import MonteCarloPricer
from pricing.pde import CrankNicolsonSolver

In [8]:
# Parameters:
S0 = 100        # Initial asset price
K = 100        # Strike
T = 1.0        # Time to maturity (1 year)
r = 0.05       # Risk-free rate
sigma = 0.2    # Volatility
option_type = "call"

In [9]:
# Solve BSM
start = time.time()
bsm_price = black_scholes_price(S0, K, T, r, sigma, option_type)
bsm_time = time.time() - start

print(f"BSM Price: {bsm_price:.4f} (Time: {bsm_time:.6f} sec)")

BSM Price: 10.4506 (Time: 0.018725 sec)


In [10]:
# Solve monte_carlo
mc = MonteCarloPricer(S0=S0,K=K,T=T,r=r,sigma=sigma,option_type=option_type,n_paths=100000,n_steps=100)
start = time.time()
mc_price = mc.price_european_option()
mc_time = time.time() - start

print(f"Monte Carlo Price: {mc_price:.4f} (Time: {mc_time:.6f} sec)")

Monte Carlo Price: 10.5179 (Time: 0.322651 sec)


In [11]:
# Initialize and solve PDE
start = time.time()
solver = CrankNicolsonSolver(S_max=200, K=K, T=T, r=r, sigma=sigma, M=100, N=100, is_call=True)
V_grid, S_grid = solver.solve()
pde_time = time.time() - start

# Interpolate the price at S
S_idx = (np.abs(S_grid - S0)).argmin()
pde_price = V_grid[S_idx]

print(f"PDE Price: {pde_price:.4f} (Time: {pde_time:.6f} sec)")

PDE Price: 10.4425 (Time: 0.031048 sec)
